**Mount Drive to Google Colab**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Step 1: Import all necessary Libraries**

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
import keras,os
from keras.layers import BatchNormalization
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import cv2
import os
import matplotlib.pyplot as plt
import zipfile

**Step 2: Give path to the zip files folder from drive,  and unzip the images and give the path**

In [ ]:
zipfile1='/content/drive/MyDrive/Computer_vision/facial_expression.zip'
unzipdirectory= '/content/drive/MyDrive/Computer_vision/'


In [ ]:
with zipfile.ZipFile(zipfile1,'r') as zip_ref:
   zip_ref.extractall(unzipdirectory)

**Step 3: Read an image from train and test dataset to check the data is unzip successfully**

In [ ]:
read_image_train=cv2.imread('/content/drive/MyDrive/Computer_vision/Dataset/train/happiness/100016214.png')
read_image_train.shape

# image dimension batch size for trainig the model

(48, 48, 3)

In [ ]:
read_image_test=cv2.imread('/content/drive/MyDrive/Computer_vision/Dataset/test/surprise/103794288.png')
read_image_test.shape

# image dimension and batch size for testing the model

(48, 48, 3)

**Step 4: Load and preprocess the data, including data augmentation**

In [ ]:
#your dataset directory containing images of 7 classes.
train_data_dir = r'/content/drive/MyDrive/Computer_vision/Dataset/train/'

# Image dimensions
img_width, img_height = 48, 48

# Batch size for training and validation
batch_size = 64

# Create data augmentation for training images
train_datagen = ImageDataGenerator(
    rescale=1./255,          # Normalize pixel values to [0, 1]
    shear_range=0.1,         # Shear transformations
    rotation_range=10,       # Degree range for random rotations
    zoom_range=0.1,          # Zoom transformations
    width_shift_range=0.1,   # Randomly shift images horizontally by 10% of the width
    height_shift_range=0.1,  # Randomly shift images vertically by 10% of the height
    vertical_flip=True,      # Randomly flip images vertically
    horizontal_flip=True,    # Randomly flip images horizontally
    fill_mode='nearest'      # Rescale pixel values to [0, 1]
)



In [ ]:
# Initialize image data generator with rescaling

validation_datagen=ImageDataGenerator(rescale=1./255)

**Data Augmentataion**

In [ ]:
# Load and preprocess training data using the data augmentation generator
train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/Computer_vision/Dataset/train',
    target_size=(img_width, img_height),
    batch_size=batch_size,
    color_mode="grayscale",
    class_mode='categorical',
    shuffle='True'
)

# Load and preprocess test data using the data augmentation generator
validation_generator = validation_datagen.flow_from_directory(
    '/content/drive/MyDrive/Computer_vision/Dataset/test',
    target_size=(img_width, img_height),
    batch_size=batch_size,
    color_mode="grayscale",
    class_mode='categorical',
    shuffle='True'
)

Found 32298 images belonging to 7 classes.
Found 3589 images belonging to 7 classes.


**Step 6: Creating CNN Model Architecture**

In [ ]:
model = models.Sequential()

In [ ]:
# Convolutional and pooling layers
model.add(layers.Conv2D(32, (3, 3),padding='valid', activation='relu', input_shape=(img_width, img_height, 1)))
model.add(BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.25))

model.add(layers.Conv2D(64, (3, 3), padding='valid',activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(128, (3, 3), padding='valid',activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.25))

# Flatten the 3D feature maps to 1D
model.add(layers.Flatten())

# Dense layers
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(7, activation='softmax'))  # 7 output classes
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 46, 46, 32)        320       
                                                                 
 batch_normalization (BatchN  (None, 46, 46, 32)       128       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 23, 23, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 23, 23, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 21, 21, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 10, 10, 64)       0

**for valid padding**
Total params: 233,735
Trainable params: 233,671
Non-trainable params: 64

**for same padding**
Total params: 397,575
Trainable params: 397,511
Non-trainable params: 64

**Step: 7 compile the model**

In [ ]:

model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy'])

**Step 8: Train the Model**

In [ ]:
from keras.engine import sequential
# Number of training samples and steps per epoch
num_train_samples = len(train_generator.filenames)
steps_per_epoch = num_train_samples // batch_size     #32298 // 64

# Number of epochs for training
epochs = 50

# Train the model
train_model = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=epochs
)

Epoch 1/50
407/504 [=======================>......] - ETA: 33:58 - loss: 1.8126 - accuracy: 0.2472

**Step 10: Save the Model**

In [ ]:

test_loss, test_acc =model.evaluate(validation_generator, verbose=2)

print('\nTest accuracy:', test_acc)
model.save('emotion_detection_model_50epochs.h5')


**Step 11: Plot the training and validation accuracy and loss at each epoch**

In [ ]:
emotion_model = train_model

In [ ]:
loss = emotion_model.history['loss']
val_loss = emotion_model.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'y', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

acc = emotion_model.history['accuracy']
#acc = history.history['accuracy']
val_acc = emotion_model.history['val_accuracy']
#val_acc = history.history['val_accuracy']

plt.plot(epochs, acc, 'y', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
from keras.models import load_model
import numpy as np
my_model = load_model('emotion_detection_model_50epochs.h5', compile=False)

#Generate a batch of images
test_img, test_lbl = validation_generator.__next__()
predictions=my_model.predict(test_img)

predictions = np.argmax(predictions, axis=1)
test_labels = np.argmax(test_lbl, axis=1)

In [ ]:

from sklearn import metrics
from sklearn import metrics
import seaborn as sns
import random
print ("Accuracy = ", metrics.accuracy_score(test_labels, predictions))

#Confusion Matrix - verify accuracy of each class
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(test_labels, predictions)
#print(cm)
import seaborn as sns
sns.heatmap(cm, annot=True)

class_labels=['Angry','Disgust', 'Fear', 'Happy','Neutral','Sad','Surprise']
#Check results on a few select images
n=random.randint(0, test_img.shape[1] - 1)
image = test_img[n]
orig_labl = class_labels[test_labels[n]]
pred_labl = class_labels[predictions[n]]
plt.imshow(image[:,:,0], cmap='gray')
plt.title("Original label is:"+orig_labl+" Predicted is: "+ pred_labl)
plt.show()

**Save in a jason file**

In [ ]:
#save in a jason file
model_json=EmotionModel.to_json()
with open("emotion_model.json","w")as json_file:
  json_file.write(model_json)

**#now saving trained model weights**

In [ ]:


EmotionModel.save_weights('emotion_model.h5')
